<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
# generación de predicción por tipo de vehículo y tipo de mantenimiento
import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import pandas.tseries.offsets as offsets
import os
from sqlalchemy import create_engine, text

# 🔧 Configuración de conexión a MySQL
DB_CONFIG = {
    "host": "roundhouse.proxy.rlwy.net",
    "port": 38517,
    "user": "root",
    "password": "wZGotyxIDqpAtQjPxNuxxezSbbroztiw",
    "database": "railway"
}

# 📥 Consulta SQL
query = """
SELECT 
    FechaElaboracion,
    Debito,
    TipoMantenimiento,
    Categoria,
    TipoMatricula,
    NombreVehiculo,
    IdentificacionTercero
FROM Hechos_Mantenimiento
WHERE Debito IS NOT NULL 
  AND FechaElaboracion IS NOT NULL
  AND TipoMantenimiento IS NOT NULL
  AND Categoria IS NOT NULL
  AND TipoMatricula IS NOT NULL
  AND NombreVehiculo IS NOT NULL
  AND IdentificacionTercero IS NOT NULL
"""

# 📊 Cargar datos
conn = mysql.connector.connect(**DB_CONFIG)
df = pd.read_sql(query, conn)
conn.close()
df['FechaElaboracion'] = pd.to_datetime(df['FechaElaboracion'])

# 📂 Crear carpeta de salida
os.makedirs("graficas_rf_vehiculo", exist_ok=True)
resultados = []

# 🔁 Por cada vehículo y tipo de mantenimiento
for vehiculo in df['NombreVehiculo'].unique():
    df_vehiculo = df[df['NombreVehiculo'] == vehiculo]
    for tipo in df_vehiculo['TipoMantenimiento'].unique():
        df_tipo = df_vehiculo[df_vehiculo['TipoMantenimiento'] == tipo].copy()
        if df_tipo.empty:
            continue

        # Agrupar por mes
        df_tipo['AñoMes'] = df_tipo['FechaElaboracion'].dt.to_period('M')
        df_hist_grouped = df_tipo.groupby('AñoMes').agg({
            'Debito': 'sum',
            'Categoria': lambda x: x.mode()[0],
            'TipoMatricula': lambda x: x.mode()[0],
            'IdentificacionTercero': lambda x: x.mode()[0]
        }).reset_index()

        df_hist_grouped['FechaElaboracion'] = df_hist_grouped['AñoMes'].dt.to_timestamp()
        df_hist_grouped['Año'] = df_hist_grouped['FechaElaboracion'].dt.year
        df_hist_grouped['Mes'] = df_hist_grouped['FechaElaboracion'].dt.month
        df_hist_grouped['TipoMantenimiento'] = tipo
        df_hist_grouped['NombreVehiculo'] = vehiculo

        # Entrenamiento del modelo
        X = df_hist_grouped[['Año', 'Mes', 'TipoMantenimiento', 'Categoria', 'TipoMatricula', 'NombreVehiculo', 'IdentificacionTercero']]
        y = df_hist_grouped['Debito']

        preprocessor = ColumnTransformer([
            ('num', StandardScaler(), ['Año', 'Mes']),
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['TipoMantenimiento', 'Categoria', 'TipoMatricula', 'NombreVehiculo', 'IdentificacionTercero'])
        ])

        pipeline = Pipeline([
            ('preprocessing', preprocessor),
            ('regresion', RandomForestRegressor(n_estimators=100, random_state=42))
        ])

        pipeline.fit(X, y)
        y_pred = pipeline.predict(X)
        r2 = r2_score(y, y_pred)

        # Generar 12 meses futuros
        ultima_fecha = df_hist_grouped['FechaElaboracion'].max()
        fechas_futuras = pd.date_range(start=ultima_fecha + offsets.MonthBegin(1), periods=12, freq='MS')

        categoria = df_hist_grouped['Categoria'].mode()[0]
        tipomatricula = df_hist_grouped['TipoMatricula'].mode()[0]
        tercero = df_hist_grouped['IdentificacionTercero'].mode()[0]

        df_futuro = pd.DataFrame({
            'Año': fechas_futuras.year,
            'Mes': fechas_futuras.month,
            'TipoMantenimiento': tipo,
            'Categoria': categoria,
            'TipoMatricula': tipomatricula,
            'NombreVehiculo': vehiculo,
            'IdentificacionTercero': tercero,
            'FechaElaboracion': fechas_futuras
        })

        y_futuro = pipeline.predict(df_futuro.drop(columns='FechaElaboracion'))
        df_futuro['Debito'] = y_futuro
        df_futuro['Prediccion'] = y_futuro
        df_futuro['R2'] = r2
        df_futuro['Origen'] = 'Predicción'

        df_hist_grouped['Prediccion'] = y_pred
        df_hist_grouped['R2'] = r2
        df_hist_grouped['Origen'] = 'Histórico'

        df_resultado = pd.concat([
            df_hist_grouped[['FechaElaboracion', 'Debito', 'Prediccion', 'R2', 'Origen', 'NombreVehiculo', 'TipoMantenimiento']],
            df_futuro[['FechaElaboracion', 'Debito', 'Prediccion', 'R2', 'Origen', 'NombreVehiculo', 'TipoMantenimiento']]
        ]).sort_values(by='FechaElaboracion')

        resultados.append(df_resultado)

        # 📈 Generar gráfico
        plt.figure(figsize=(14, 5))
        df_h = df_resultado[df_resultado['Origen'] == 'Histórico']
        df_p = df_resultado[df_resultado['Origen'] == 'Predicción']

        plt.plot(df_h['FechaElaboracion'], df_h['Debito'], marker='o', linestyle='-', label='Histórico')
        plt.plot(df_p['FechaElaboracion'], df_p['Prediccion'], marker='x', linestyle='--', color='orange', label='Predicción')

        plt.title(f"{vehiculo} - Tipo: {tipo}")
        plt.xlabel("Fecha")
        plt.ylabel("Costo (Débito)")
        plt.grid(True)
        plt.legend()
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
        plt.xticks(rotation=45)
        plt.text(0.98, 0.02, f"R² = {r2:.4f}", transform=plt.gca().transAxes,
                 fontsize=10, ha='right', va='bottom',
                 bbox=dict(facecolor='white', alpha=0.6, edgecolor='gray'))
        plt.tight_layout()

        file_name = f"graficas_rf_vehiculo/{vehiculo.replace(' ', '_')}_{tipo.replace(' ', '_')}.png"
        plt.savefig(file_name)
        plt.close()

# 📤 Exportar Excel
df_final = pd.concat(resultados, ignore_index=True)
df_final.to_excel("Prediccion_Costos_RF_Por_Vehiculo_Tipo.xlsx", index=False)

# 📥 Insertar en la tabla MySQL: Predicciones_Vehiculo_Tipo
df_insert = df_final[['NombreVehiculo', 'TipoMantenimiento', 'FechaElaboracion', 'Debito', 'Origen']].copy()
df_insert.columns = ['NombreVehiculo', 'TipoMantenimiento', 'Fecha', 'Costo', 'Origen']
df_insert = df_insert.dropna(subset=['Costo'])
df_insert['Costo'] = df_insert['Costo'].round(2)

# Conexión SQLAlchemy
DB_URL = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(DB_URL)

with engine.begin() as conn:
    for _, row in df_insert.iterrows():
        query_insert = text("""
            INSERT INTO Predicciones_Vehiculo_Tipo (NombreVehiculo, TipoMantenimiento, Fecha, Costo, Origen)
            VALUES (:NombreVehiculo, :TipoMantenimiento, :Fecha, :Costo, :Origen)
            ON DUPLICATE KEY UPDATE Costo = VALUES(Costo)
        """)
        conn.execute(query_insert, {
            "NombreVehiculo": row['NombreVehiculo'],
            "TipoMantenimiento": row['TipoMantenimiento'],
            "Fecha": row['Fecha'],
            "Costo": row['Costo'],
            "Origen": row['Origen']
        })


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# generación de predicción general por tipo de mantenimiento
import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from datetime import datetime
import pandas.tseries.offsets as offsets
import os
from sqlalchemy import create_engine, text

# 🔧 CONEXIÓN A LA BASE DE DATOS
DB_CONFIG = {
    "host": "roundhouse.proxy.rlwy.net",
    "port": 38517,
    "user": "root",
    "password": "wZGotyxIDqpAtQjPxNuxxezSbbroztiw",
    "database": "railway"
}

# 📥 CONSULTA
query = """
SELECT 
    FechaElaboracion,
    Debito,
    TipoMantenimiento,
    Categoria,
    TipoMatricula,
    NombreVehiculo,
    IdentificacionTercero
FROM Hechos_Mantenimiento
WHERE Debito IS NOT NULL 
  AND FechaElaboracion IS NOT NULL
  AND TipoMantenimiento IS NOT NULL
  AND Categoria IS NOT NULL
  AND TipoMatricula IS NOT NULL
  AND NombreVehiculo IS NOT NULL
  AND IdentificacionTercero IS NOT NULL
"""

# 📊 CARGAR DATOS
conn = mysql.connector.connect(**DB_CONFIG)
df = pd.read_sql(query, conn)
conn.close()

df['FechaElaboracion'] = pd.to_datetime(df['FechaElaboracion'])

# 📂 CARPETA PARA GRÁFICAS
os.makedirs("graficas_rf_agrupado", exist_ok=True)
resultados = []

# 🔁 POR CADA TIPO DE MANTENIMIENTO
for tipo in df['TipoMantenimiento'].unique():
    df_tipo = df[df['TipoMantenimiento'] == tipo].copy()

    # 📆 AGRUPAR POR MES
    df_tipo['AñoMes'] = df_tipo['FechaElaboracion'].dt.to_period('M')
    df_hist_grouped = df_tipo.groupby('AñoMes').agg({
        'Debito': 'sum',
        'Categoria': lambda x: x.mode()[0],
        'TipoMatricula': lambda x: x.mode()[0],
        'NombreVehiculo': lambda x: x.mode()[0],
        'IdentificacionTercero': lambda x: x.mode()[0]
    }).reset_index()

    df_hist_grouped['FechaElaboracion'] = df_hist_grouped['AñoMes'].dt.to_timestamp()
    df_hist_grouped['Año'] = df_hist_grouped['FechaElaboracion'].dt.year
    df_hist_grouped['Mes'] = df_hist_grouped['FechaElaboracion'].dt.month
    df_hist_grouped['TipoMantenimiento'] = tipo

    # 🎓 ENTRENAR MODELO
    X = df_hist_grouped[['Año', 'Mes', 'TipoMantenimiento', 'Categoria', 'TipoMatricula', 'NombreVehiculo', 'IdentificacionTercero']]
    y = df_hist_grouped['Debito']

    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), ['Año', 'Mes']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['TipoMantenimiento', 'Categoria', 'TipoMatricula', 'NombreVehiculo', 'IdentificacionTercero'])
    ])

    pipeline = Pipeline([
        ('preprocessing', preprocessor),
        ('regresion', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    pipeline.fit(X, y)
    y_pred = pipeline.predict(X)
    r2 = r2_score(y, y_pred)

    # 📅 GENERAR 12 MESES FUTUROS
    ultima_fecha = df_hist_grouped['FechaElaboracion'].max()
    fechas_futuras = pd.date_range(start=ultima_fecha + offsets.MonthBegin(1), periods=12, freq='MS')

    categoria = df_hist_grouped['Categoria'].mode()[0]
    tipomatricula = df_hist_grouped['TipoMatricula'].mode()[0]
    vehiculo = df_hist_grouped['NombreVehiculo'].mode()[0]
    tercero = df_hist_grouped['IdentificacionTercero'].mode()[0]

    df_futuro = pd.DataFrame({
        'Año': fechas_futuras.year,
        'Mes': fechas_futuras.month,
        'TipoMantenimiento': tipo,
        'Categoria': categoria,
        'TipoMatricula': tipomatricula,
        'NombreVehiculo': vehiculo,
        'IdentificacionTercero': tercero,
        'FechaElaboracion': fechas_futuras
    })

    y_futuro = pipeline.predict(df_futuro.drop(columns='FechaElaboracion'))
    df_futuro['Debito'] = y_futuro
    df_futuro['Prediccion'] = y_futuro
    df_futuro['R2'] = r2
    df_futuro['Origen'] = 'Predicción'

    df_hist_grouped['Prediccion'] = y_pred
    df_hist_grouped['R2'] = r2
    df_hist_grouped['Origen'] = 'Histórico'

    df_resultado = pd.concat([
        df_hist_grouped[['FechaElaboracion', 'Debito', 'Prediccion', 'R2', 'Origen', 'TipoMantenimiento']],
        df_futuro[['FechaElaboracion', 'Debito', 'Prediccion', 'R2', 'Origen', 'TipoMantenimiento']]
    ]).sort_values(by='FechaElaboracion')

    resultados.append(df_resultado)

    # 📈 GRAFICAR
    plt.figure(figsize=(14, 5))
    df_h = df_resultado[df_resultado['Origen'] == 'Histórico']
    df_p = df_resultado[df_resultado['Origen'] == 'Predicción']

    plt.plot(df_h['FechaElaboracion'], df_h['Debito'], marker='o', linestyle='-', label='Histórico')
    plt.plot(df_p['FechaElaboracion'], df_p['Prediccion'], marker='x', linestyle='--', color='orange', label='Predicción')

    plt.title(f"Costo Mantenimiento - Tipo: {tipo}")
    plt.xlabel("Fecha")
    plt.ylabel("Costo (Débito)")
    plt.grid(True)
    plt.legend()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    plt.xticks(rotation=45)

    plt.text(
        0.98, 0.02, f"R² = {r2:.4f}",
        transform=plt.gca().transAxes,
        fontsize=10, ha='right', va='bottom',
        bbox=dict(facecolor='white', alpha=0.6, edgecolor='gray')
    )

    plt.tight_layout()
    archivo = f"graficas_rf_agrupado/Modelo_RF_{tipo.replace(' ', '_')}.png"
    plt.savefig(archivo)
    plt.close()

# 💾 EXPORTAR A EXCEL
df_final = pd.concat(resultados, ignore_index=True)
df_final.to_excel("Prediccion_Costos_RF_Mensual_Agrupado.xlsx", index=False)

# 📥 INSERTAR EN MYSQL: Predicciones_Tipo_Mantenimiento
df_insert = df_final[['TipoMantenimiento', 'FechaElaboracion', 'Debito', 'Origen']].copy()
df_insert.columns = ['TipoMantenimiento', 'Fecha', 'Costo', 'Origen']
df_insert = df_insert.dropna(subset=['Costo'])
df_insert['Costo'] = df_insert['Costo'].round(2)

# 🔌 Conexión con SQLAlchemy
DB_URL = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(DB_URL)

with engine.begin() as conn:
    for _, row in df_insert.iterrows():
        query_insert = text("""
            INSERT INTO Predicciones_Tipo_Mantenimiento (TipoMantenimiento, Fecha, Costo, Origen)
            VALUES (:TipoMantenimiento, :Fecha, :Costo, :Origen)
            ON DUPLICATE KEY UPDATE Costo = VALUES(Costo)
        """)
        conn.execute(query_insert, {
            "TipoMantenimiento": row['TipoMantenimiento'],
            "Fecha": row['Fecha'],
            "Costo": row['Costo'],
            "Origen": row['Origen']
        })
